In [ ]:
#데이터분할 
#2회 기출, 3회기출, 4회기출
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train[COL_NUM+COL_CAT]
                                            , y_train[COL_Y].values.ravel()
                                            , test_size=0.3
                                            , stratify=y_train[COL_Y].values.ravel())



In [ ]:
#데이터스케일링
#2회기출, 3회기출
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr[COL_NUM])

X_tr[COL_NUM]   = scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM]  = scaler.transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.transform(X_test[COL_NUM])

In [ ]:
#범주형데이터
#2회기출
from sklearn.preprocessing import LabelEncoder

X = pd.concat([X_train[COL_CAT], X_test[COL_CAT]])

for col in COL_CAT:
  le = LabelEncoder()
  le.fit(X[col])
  X_tr[col] = le.transform(X_tr[col])
  X_val[col] = le.transform(X_val[col])
  X_test[col] = le.transform(X_test[col])

  # 각 변수의 클래스 확인
  print(col)
  print(le.classes_)
  print('\n')

In [ ]:
#모형 생성
#2회기출
## 1) 랜덤 포레스트
from sklearn.ensemble import RandomForestClassifier
modelRF = RandomForestClassifier(random_state=123)
modelRF.fit(X_tr, y_tr)

## 2) XGBoost
from xgboost import XGBClassifier
modelXGB = XGBClassifier(random_state=123)
modelXGB.fit(X_tr, y_tr)

## 1) 랜덤 포레스트
from sklearn.ensemble import RandomForestRegressor
modelRF = RandomForestRegressor(random_state=123)
modelRF.fit(X_tr, y_tr)

## 2) XGBoost
from xgboost import XGBRegressor
modelXGB = XGBRegressor(objective='reg:squarederror', random_state=123)
modelXGB.fit(X_tr, y_tr)

In [ ]:
#모형평가
#2회기출
## 답안 채점 기준인 roc_auc 사용
from sklearn.metrics import roc_auc_score

scoreRF = roc_auc_score(y_val, y_val_predRF)
scoreXGB = roc_auc_score(y_val, y_val_predXGB)

print('Random Forest: \t', scoreRF)
print('XGBoost: \t', scoreXGB)

#4회기출
## 답안 채점 기준인 macro f1 score 사용
from sklearn.metrics import f1_score

scoreRF = f1_score(y_val, y_val_predRF, average='macro')
scoreXGB = f1_score(y_val, y_val_predXGB, average='macro')

print('Random Forest: \t', scoreRF)
print('XGBoost: \t', scoreXGB)

## 답안 채점 기준인 rmse 사용 (mean_squared_error를 사용하여 검증해도 됩니다.)
from sklearn.metrics import mean_squared_error
import numpy as np

def cal_rmse(actual, pred):
  return np.sqrt(mean_squared_error(actual, pred))

scoreRF = cal_rmse(y_val, y_val_predRF)
scoreXGB = cal_rmse(y_val, y_val_predXGB)

print('Random Forest: \t', scoreRF)
print('XGBoost: \t', scoreXGB)


In [88]:
import pandas as pd
X_test = pd.read_csv("yemoonsaBigdata/datasets/Part3/504_x_test.csv")
X_train = pd.read_csv("yemoonsaBigdata/datasets/Part3/504_x_train.csv")
y_train = pd.read_csv("yemoonsaBigdata/datasets/Part3/504_y_train.csv")

##데이터 전처리
# print(X_train.head())
# print(X_train.info())
# print(X_train.isnull().sum())

# print(X_train.columns)
##변수설정
COL_DEL = ['id']
COL_NUM = ['year', 'mileage', 'tax', 'mpg', 'engineSize']
COL_CAT = ['model', 'transmission', 'fuelType']
COL_Y =['price']

##데이터분리
from sklearn.model_selection import train_test_split

X_tr,X_var,y_tr,y_var = train_test_split(X_train[COL_NUM+COL_CAT],
                                        y_train[COL_Y].values.ravel(),
                                        test_size = 0.3,
                                        )

##데이터 스케일링
from sklearn.preprocessing import StandardScaler
std = StandardScaler()

std.fit(X_test[COL_NUM])
X_tr[COL_NUM] = std.transform(X_tr[COL_NUM])
X_var[COL_NUM] = std.transform(X_var[COL_NUM])
X_test[COL_NUM] = std.transform(X_test[COL_NUM])

#데이터 범주화
from sklearn.preprocessing import LabelEncoder

for _col in COL_CAT:
    le = LabelEncoder()
    le.fit(X_train[_col])
    X_tr[_col] =  le.transform(X_tr[_col])
    X_var[_col] =  le.transform(X_var[_col])
    X_test[_col] =  le.transform(X_test[_col])
    # print(_col)
    # print(le.classes_)

##모형생성
###랜덤포레스트
from sklearn.ensemble import RandomForestRegressor
modelRF = RandomForestRegressor(n_estimators=123)
modelRF.fit(X_tr,y_tr)


###부스팅
from xgboost import XGBRegressor
modelXGB = XGBRegressor(n_estimators=123)
modelXGB.fit(X_tr,y_tr)

##모형평가
from sklearn.metrics import mean_squared_error
import numpy as np

y_pred_RF = modelRF.predict(X_var)
y_pred_XGB = modelXGB.predict(X_var)

# print(np.sqrt(mean_squared_error(y_pred_RF, y_var)))
# print(np.sqrt(mean_squared_error(y_pred_XGB, y_var)))

##모형평가함수
def get_scort_rmse(model,X_tr,X_var,y_tr, y_var):
    y_pred_tr = model.predict(X_tr)
    y_pred_var = model.predict(X_var)

    rmse_tr = np.sqrt(mean_squared_error(y_pred_tr, y_tr))
    rmse_var = np.sqrt(mean_squared_error(y_pred_var, y_var))

    return rmse_tr, rmse_var 

##결과값 출력 
#램덤포레스트
rmse_tr_RF, rmse_var_RF = get_scort_rmse(modelRF,X_tr,X_var,y_tr, y_var)
# print(f'RF_tr : {rmse_tr_RF}, RF_var : {rmse_var_RF}')

#부스팅
rmse_tr_XGB, rmse_var_XGB = get_scort_rmse(modelXGB,X_tr,X_var,y_tr, y_var)
# print(f'XG_tr : {rmse_tr_XGB}, XG_var : {rmse_var_XGB}')

##예측값 생성
pred_y = modelRF.predict(X_test[COL_NUM+COL_CAT])
pred_y = [int(x) for x in pred_y]
result = pd.DataFrame({'id' : X_test['id'],'price' : pred_y})

result.to_csv('0000000.csv',index=False)
result



,id,price
0,7469,34373
1,7470,16813
2,7471,34979
3,7472,15943
4,7473,16274
...,...,...
3195,10664,17239
3196,10665,10876
3197,10666,18042
3198,10667,25184


In [126]:
import pandas as pd
import pandas as pd

X_test = pd.read_csv("yemoonsaBigdata/datasets/Part3/204_x_test.csv")
X_train = pd.read_csv("yemoonsaBigdata/datasets/Part3/204_x_train.csv")
y_train = pd.read_csv("yemoonsaBigdata/datasets/Part3/204_y_train.csv")
##데이터 전처리
# print(X_train.info())
# print(X_train.isnull().sum())

##데이터 변수 설정 
# print(X_train.head())
# print(y_train)
COL_DEL = ['ID']
COL_NUM = ['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 'Discount_offered', 'Weight_in_gms']
COL_CAT = ['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']
COL_Y = ['Reached.on.Time_Y.N']

##테스트 변수분리
from sklearn.model_selection import train_test_split
X_tr,X_var,y_tr,y_var = train_test_split(X_train[COL_NUM+COL_CAT],
                                         y_train[COL_Y].values.ravel(),
                                         test_size = 0.3)

##수치형 변수 스케일링
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
std.fit(X_train[COL_NUM])

X_tr[COL_NUM] = std.transform(X_tr[COL_NUM])
X_var[COL_NUM] = std.transform(X_var[COL_NUM])
X_test[COL_NUM] = std.transform(X_test[COL_NUM])

##범주형 변수 범주화
from sklearn.preprocessing import LabelEncoder

for _col in COL_CAT:
    le = LabelEncoder()
    le.fit(X_train[_col])

    X_tr[_col] = le.transform(X_tr[_col])
    X_var[_col] = le.transform(X_var[_col])
    X_test[_col] = le.transform(X_test[_col])
    # print(_col)
    # print(le.classes_)

##모델생성
#랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
modelRF = RandomForestClassifier(n_estimators=123)
modelRF.fit(X_tr,y_tr)

from xgboost import XGBClassifier
modelXG = XGBClassifier()
modelXG.fit(X_tr,y_tr)


X_test = pd.read_csv("yemoonsaBigdata/datasets/Part3/204_x_test.csv")
X_train = pd.read_csv("yemoonsaBigdata/datasets/Part3/204_x_train.csv")
y_train = pd.read_csv("yemoonsaBigdata/datasets/Part3/204_y_train.csv")
##데이터 전처리
# print(X_train.info())
# print(X_train.isnull().sum())

##데이터 변수 설정 
# print(X_train.head())
# print(y_train)
COL_DEL = ['ID']
COL_NUM = ['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 'Discount_offered', 'Weight_in_gms']
COL_CAT = ['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']
COL_Y = ['Reached.on.Time_Y.N']

##테스트 변수분리
from sklearn.model_selection import train_test_split
X_tr,X_var,y_tr,y_var = train_test_split(X_train[COL_NUM+COL_CAT],
                                         y_train[COL_Y].values.ravel(),
                                         test_size = 0.3)

##수치형 변수 스케일링
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
std.fit(X_train[COL_NUM])

X_tr[COL_NUM] = std.transform(X_tr[COL_NUM])
X_var[COL_NUM] = std.transform(X_var[COL_NUM])
X_test[COL_NUM] = std.transform(X_test[COL_NUM])

##범주형 변수 범주화
from sklearn.preprocessing import LabelEncoder

for _col in COL_CAT:
    le = LabelEncoder()
    le.fit(X_train[_col])

    X_tr[_col] = le.transform(X_tr[_col])
    X_var[_col] = le.transform(X_var[_col])
    X_test[_col] = le.transform(X_test[_col])
    # print(_col)
    # print(le.classes_)

##모델생성
#랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
modelRF = RandomForestClassifier(n_estimators=123)
modelRF.fit(X_tr,y_tr)

from xgboost import XGBClassifier
modelXG = XGBClassifier()
modelXG.fit(X_tr,y_tr)
    
##모델평가
from sklearn.metrics import roc_auc_score
y_pred_RF = modelRF.predict_proba(X_var)[:,1]
y_pred_XG = modelXG.predict_proba(X_var)[:,1]
print(roc_auc_score(y_var,y_pred_RF))
print(roc_auc_score(y_var,y_pred_XG)

##

0.7417530461816707